# ECS764P Applied Statistics - Coursework 2

- **Professors:** Dr. Frederik Dahlqvist
- **Due Date:** Thursday, 15. January 2026 (12:00 GMT)
- **Student:** Philipp Schmidt
- **Python Version:** 3.11.14


In [ ]:
# Download the required dependencies
import os

if os.path.exists('requirements.txt'):
    %pip install -q -r requirements.txt
else:
    print("requirements.txt not found. Installing packages manually...")
    %pip install -q matplotlib numpy scipy
